In [ ]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import keras
import keras_nlp
import tensorflow_datasets as tfds

In [ ]:
snli_train = tfds.load("snli", split="train[:20%]")
snli_val = tfds.load("snli", split="validation")
snli_test = tfds.load("snli", split="test")

# Here's an example of how our training samples look like, where we randomly select
# four samples:
sample = snli_test.batch(4).take(1).get_single_element()
sample

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Shuffling /root/tensorflow_datasets/snli/incomplete.YINKIZ_1.1.0/snli-validation.tfrecord*...:   0%|          …

Generating train examples...:   0%|          | 0/550152 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/snli/incomplete.YINKIZ_1.1.0/snli-train.tfrecord*...:   0%|          | 0/5…

Dataset snli downloaded and prepared to /root/tensorflow_datasets/snli/1.1.0. Subsequent calls will reuse this data.


{'hypothesis': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl is entertaining on stage',
        b'A group of people posing in front of a body of water.',
        b"The group of people aren't inide of the building.",
        b'The people are taking a carriage ride.'], dtype=object)>,
 'label': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>,
 'premise': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl in a blue leotard hula hoops on a stage with balloon shapes in the background.',
        b'A group of people taking pictures on a walkway in front of a large body of water.',
        b'Many people standing outside of a place talking to each other in front of a building that has a sign that says "HI-POINTE."',
        b'Three people are riding a carriage pulled by four horses.'],
       dtype=object)>}

In [ ]:
def filter_labels(sample):
    return sample["label"] >= 0

In [ ]:
def split_labels(sample):
    x = (sample["hypothesis"], sample["premise"])
    y = sample["label"]
    return x, y


train_ds = (
    snli_train.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)
val_ds = (
    snli_val.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)
test_ds = (
    snli_test.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)

In [ ]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)

100%|██████████| 507/507 [00:00<00:00, 1.13MB/s]


100%|██████████| 16.8M/16.8M [00:02<00:00, 6.31MB/s]


100%|██████████| 547/547 [00:00<00:00, 577kB/s]


100%|██████████| 226k/226k [00:00<00:00, 277kB/s]


In [ ]:
bert_classifier.fit(train_ds, validation_data=val_ds, epochs=3)


Epoch 1/3
   6867/Unknown 180s 24ms/step - loss: 0.8744 - sparse_categorical_accuracy: 0.5881

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6867/6867 ━━━━━━━━━━━━━━━━━━━━ 191s 25ms/step - loss: 0.8744 - sparse_categorical_accuracy: 0.5881 - val_loss: 0.5859 - val_sparse_categorical_accuracy: 0.7598
Epoch 2/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 174s 23ms/step - loss: 0.6531 - sparse_categorical_accuracy: 0.7283 - val_loss: 0.5453 - val_sparse_categorical_accuracy: 0.7787
Epoch 3/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 155s 22ms/step - loss: 0.5954 - sparse_categorical_accuracy: 0.7581 - val_loss: 0.5306 - val_sparse_categorical_accuracy: 0.7900


In [ ]:
bert_classifier.evaluate(test_ds)


614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.5221 - sparse_categorical_accuracy: 0.7958


[0.5344147086143494, 0.7879682183265686]

In [ ]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)
bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=["accuracy"],
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=1)
bert_classifier.evaluate(test_ds)

6867/6867 ━━━━━━━━━━━━━━━━━━━━ 188s 25ms/step - accuracy: 0.6101 - loss: 0.8527 - val_accuracy: 0.7582 - val_loss: 0.5881
614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7659 - loss: 0.5834


[0.5908031463623047, 0.7614006400108337]

In [ ]:
class TriangularSchedule(keras.optimizers.schedules.LearningRateSchedule):
    """Linear ramp up for `warmup` steps, then linear decay to zero at `total` steps."""

    def __init__(self, rate, warmup, total):
        self.rate = rate
        self.warmup = warmup
        self.total = total

    def get_config(self):
        config = {"rate": self.rate, "warmup": self.warmup, "total": self.total}
        return config

    def __call__(self, step):
        step = keras.ops.cast(step, dtype="float32")
        rate = keras.ops.cast(self.rate, dtype="float32")
        warmup = keras.ops.cast(self.warmup, dtype="float32")
        total = keras.ops.cast(self.total, dtype="float32")

        warmup_rate = rate * step / self.warmup
        cooldown_rate = rate * (total - step) / (total - warmup)
        triangular_rate = keras.ops.minimum(warmup_rate, cooldown_rate)
        return keras.ops.maximum(triangular_rate, 0.0)


bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)

# Get the total count of training batches.
# This requires walking the dataset to filter all -1 labels.
epochs = 3
total_steps = sum(1 for _ in train_ds.as_numpy_iterator()) * epochs
warmup_steps = int(total_steps * 0.2)

bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(
        TriangularSchedule(1e-4, warmup_steps, total_steps)
    ),
    metrics=["accuracy"],
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 192s 25ms/step - accuracy: 0.5312 - loss: 0.9309 - val_accuracy: 0.7585 - val_loss: 0.5887
Epoch 2/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 177s 23ms/step - accuracy: 0.7326 - loss: 0.6444 - val_accuracy: 0.7851 - val_loss: 0.5349
Epoch 3/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 204s 23ms/step - accuracy: 0.7699 - loss: 0.5699 - val_accuracy: 0.7905 - val_loss: 0.5210


In [ ]:
bert_classifier.evaluate(test_ds)


614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7907 - loss: 0.5174


[0.5280759334564209, 0.7883753776550293]

In [ ]:
bert_classifier.save("bert_classifier.keras")
restored_model = keras.models.load_model("bert_classifier.keras")
restored_model.evaluate(test_ds)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 83 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


614/614 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.5174 - sparse_categorical_accuracy: 0.7907


[0.5280759334564209, 0.7883753776550293]

In [ ]:
# Convert to Hypothesis-Premise pair, for forward pass through model
sample = (sample["hypothesis"], sample["premise"])
sample

(<tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl is entertaining on stage',
        b'A group of people posing in front of a body of water.',
        b"The group of people aren't inide of the building.",
        b'The people are taking a carriage ride.'], dtype=object)>,
 <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl in a blue leotard hula hoops on a stage with balloon shapes in the background.',
        b'A group of people taking pictures on a walkway in front of a large body of water.',
        b'Many people standing outside of a place talking to each other in front of a building that has a sign that says "HI-POINTE."',
        b'Three people are riding a carriage pulled by four horses.'],
       dtype=object)>)

In [ ]:
predictions = bert_classifier.predict(sample)


def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=0)


# Get the class predictions with maximum probabilities
predictions = softmax(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
# Inittializing a RoBERTa from preset
roberta_classifier = keras_nlp.models.RobertaClassifier.from_preset(
    "roberta_base_en", num_classes=3
)

roberta_classifier.fit(train_ds, validation_data=val_ds, epochs=1)

roberta_classifier.evaluate(test_ds)

100%|██████████| 498/498 [00:00<00:00, 574kB/s]


100%|██████████| 474M/474M [00:31<00:00, 15.6MB/s]


100%|██████████| 463/463 [00:00<00:00, 868kB/s]


100%|██████████| 0.99M/0.99M [00:01<00:00, 755kB/s]


100%|██████████| 446k/446k [00:01<00:00, 428kB/s]


   3760/Unknown 5548s 1s/step - loss: 0.6551 - sparse_categorical_accuracy: 0.7226

In [ ]:
predictions = roberta_classifier.predict(sample)
print(tf.math.argmax(predictions, axis=1).numpy())